# 

In [39]:
#import sys
#print("Python executable đang dùng:", sys.executable)

#Cài thư viện ĐÚNG VÀO env của kernel hiện tại
#!"{sys.executable}" -m pip install nibabel matplotlib ipywidgets pandas
#import sys
#!"{sys.executable}" -m pip install scikit-image==0.24.0 albumentations==1.3.1 opencv-python-headless==4.10.0.84 matplotlib==3.8.4 scikit-learn==1.4.2 tqdm==4.66.5


In [40]:
from pathlib import Path
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import Dropdown, IntSlider, Button, HBox, VBox, ToggleButtons, Output, Label
from skimage import measure, morphology

# Đảm bảo notebook đặt ở C:\NCKH
BASE_DIR = Path(".").resolve()

RAW_DIR = BASE_DIR / "Data" / "raw" / "CT-Rate"
OUT_DIR = BASE_DIR / "Data" / "processed" / "CT_Rate_labeled"
OUT_DIR.mkdir(parents=True, exist_ok=True)

MANIFEST_PATH = OUT_DIR / "manifest.csv"

print("RAW_DIR:", RAW_DIR)
print("OUT_DIR:", OUT_DIR)
print("MANIFEST:", MANIFEST_PATH)


RAW_DIR: D:\TrainAI\New folder\nckh\Data\raw\CT-Rate
OUT_DIR: D:\TrainAI\New folder\nckh\Data\processed\CT_Rate_labeled
MANIFEST: D:\TrainAI\New folder\nckh\Data\processed\CT_Rate_labeled\manifest.csv


In [41]:
# Nếu đã có manifest thì load, còn không thì tạo mới
if MANIFEST_PATH.exists():
    manifest_df = pd.read_csv(MANIFEST_PATH)
else:
    manifest_df = pd.DataFrame(columns=[
        "volume_name", "slice_index", "label", "image_path"
    ])

manifest_df.head()


,volume_name,slice_index,label,image_path,data_path
0,train_1022_a_10.nii,144,CLE,NaN,Data\processed\CT_Rate_labeled\train_1022_a_10...
1,train_1022_a_10.nii,146,CLE,NaN,Data\processed\CT_Rate_labeled\train_1022_a_10...
2,train_1022_a_10.nii,145,CLE,NaN,Data\processed\CT_Rate_labeled\train_1022_a_10...
3,train_1022_a_10.nii,135,CLE,NaN,Data\processed\CT_Rate_labeled\train_1022_a_10...
4,train_1022_a_10.nii,142,CLE,NaN,Data\processed\CT_Rate_labeled\train_1022_a_10...


In [42]:
def load_volume(path: Path):
    """Đọc file .nii hoặc .nii.gz, trả về numpy array (z, y, x)."""
    img = nib.load(str(path))
    data = img.get_fdata()
    # Thông thường data shape = (x, y, z) → ta hoán vị thành (z, y, x) cho dễ duyệt
    if data.ndim == 3:
        data = np.transpose(data, (2, 1, 0))  # z, y, x
    elif data.ndim == 4:
        # Nếu có thêm channel thì lấy channel đầu
        data = np.transpose(data[..., 0], (2, 1, 0))
    return data

import numpy as np

def normalize_slice(slice_2d, hu_min, hu_max):
    """Chuẩn hóa slice theo khoảng HU [hu_min, hu_max] → 0–255."""
    s = slice_2d.astype(np.float32)
    s = np.clip(s, hu_min, hu_max)
    s = (s - hu_min) / (hu_max - hu_min + 1e-6)
    s = (s * 255).astype(np.uint8)
    return s


# ================== SEGMENT PHỔI ==================

from skimage import measure, morphology

def adaptive_lung_threshold(img_hu, lo=-400, hi=-260):
    """
    Ước lượng ngưỡng để tách phổi (không khí trong lồng ngực) với mô mềm.
    """
    torso = img_hu > -800      # lọc nền ngoài cơ thể
    vals = img_hu[torso].astype(np.float32)
    if vals.size == 0:
        # fallback an toàn
        return -320.0
    hist, bins = np.histogram(vals, bins=512)
    thr = bins[np.argmax(hist.cumsum() > hist.sum() / 2)]
    thr = np.clip(thr, lo, hi)
    return float(thr)


def largest_two_components(mask: np.ndarray) -> np.ndarray:
    labels = measure.label(mask, connectivity=2)
    if labels.max() == 0:
        return np.zeros_like(mask, dtype=bool)
    regs = measure.regionprops(labels)
    areas = sorted(((r.label, r.area) for r in regs), key=lambda x: x[1], reverse=True)
    keep = [lab for lab, _ in areas[:2]]
    return np.isin(labels, keep)


def segment_lungs_hu(img_hu: np.ndarray) -> np.ndarray:
    """
    Trích xuất lung mask 2D từ 1 lát CT (đơn vị HU):

    1) Dùng adaptive_lung_threshold để tách không khí (phổi + nền ngoài).
    2) Loại thành phần dính biên (background ngoài cơ thể).
    3) Giữ 2 thành phần lớn nhất (hai lá phổi).
    4) Làm mịn bằng morphology.
    """
    thr = adaptive_lung_threshold(img_hu)
    air_mask = img_hu < thr

    # loại background ngoài cơ thể (thành phần dính biên)
    labels = measure.label(air_mask, connectivity=2)
    h, w = air_mask.shape
    border = np.zeros_like(air_mask, dtype=bool)
    border[0, :] = True
    border[-1, :] = True
    border[:, 0] = True
    border[:, -1] = True
    border_labels = np.unique(labels[border])
    inside_mask = ~np.isin(labels, border_labels) & (labels > 0)

    lung = largest_two_components(inside_mask)
    lung = morphology.binary_closing(lung, morphology.disk(4))
    lung = morphology.binary_opening(lung, morphology.disk(2))
    lung = morphology.remove_small_holes(lung, area_threshold=1500)
    lung = morphology.remove_small_objects(lung, min_size=1500)
    return lung.astype(bool)


# ================== DÒ LÁT NGHI NGỜ EMPHYSEMA ==================

def detect_emphysema_slices(
    volume,
    hu_emph: float = -950,
    min_ratio: float = 0.13,
):
    """
    Trả về list [(slice_index, lowHU_ratio), ...] các lát nghi ngờ emphysema.

    - volume: numpy array (z, y, x) ở đơn vị HU
    - hu_emph: ngưỡng HU dưới mức này được coi là emphysema (thường -950)
    - min_ratio: tỉ lệ voxel < hu_emph trong vùng phổi để coi là "nghi ngờ"
    """
    suspicious = []

    n_slices = volume.shape[0]
    for idx in range(n_slices):
        s = volume[idx, :, :]

        # 1) Segment phổi
        lung_mask = segment_lungs_hu(s)
        lung_pixels = s[lung_mask]

        # Nếu chưa "vào" vùng phổi (pixel phổi quá ít) thì bỏ (loại hẳn các lát gần như không có phổi)
        if lung_pixels.size < 500:
            continue

        low_mask = (lung_pixels < hu_emph)
        low_count = int(np.sum(low_mask))
        lung_count = int(lung_pixels.size)
        low_hu_ratio = low_count / lung_count
        
        if low_hu_ratio >= min_ratio:
            suspicious.append((idx, low_hu_ratio, low_count, lung_count))


    return suspicious



In [43]:
# Lấy tất cả .nii và .nii.gz
volume_paths = sorted(list(RAW_DIR.glob("*.nii")) + list(RAW_DIR.glob("*.nii.gz")))

if not volume_paths:
    print("Không tìm thấy file .nii / .nii.gz trong", RAW_DIR)
else:
    print("Tìm thấy", len(volume_paths), "volume.")
    for p in volume_paths[:5]:
        print(" -", p.name)


Tìm thấy 128 volume.
 - train_10043_a_1.nii.gz
 - train_1036_a_1.nii.gz
 - train_10390_a_1.nii.gz
 - train_10636_a_1.nii.gz
 - train_1087_a_1.nii.gz


In [44]:
# --- Widgets ---

volume_dropdown = Dropdown(
    options=[(p.name, p) for p in volume_paths],
    description="Volume:",
    layout={"width": "400px"},
)

from ipywidgets import IntSlider

hu_min_slider = IntSlider(
    value=-1000, min=-1800, max=0, step=50,
    description="HU min:",
    continuous_update=False,
    layout={"width": "400px"}
)

hu_max_slider = IntSlider(
    value=200, min=-800, max=800, step=50,
    description="HU max:",
    continuous_update=False,
    layout={"width": "400px"}
)

slice_slider = IntSlider(
    value=0,
    min=0,
    max=1,   # sẽ cập nhật lại sau khi load volume
    step=1,
    description="Slice:",
    continuous_update=False,
    layout={"width": "400px"},
)

# 🔹 Dropdown cho các lát nghi ngờ emphysema
sus_slice_dropdown = Dropdown(
    options=[],   # sẽ fill sau khi detect
    description="Slice nghi ngờ:",
    layout={"width": "400px"},
)

label_buttons = ToggleButtons(
    options=["CLE", "PSE", "PLE"],
    description="Label:",
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
)

save_button = Button(
    description="Lưu slice + label",
    button_style="success",
)

info_label = Label(value="")
out = Output()

# thêm key emphy_slices để lưu kết quả detect cho volume hiện tại
current_volume_data = {
    "data": None,
    "path": None,
    "emphy_slices": [],
}


In [45]:
def on_volume_change(change):
    if change["name"] != "value" or change["new"] is None:
        return

    vol_path = change["new"]
    print(f"Đang load volume: {vol_path.name} ...")

    data = load_volume(vol_path)
    current_volume_data["data"] = data
    current_volume_data["path"] = vol_path

    # Cập nhật slider slice
    n_slices = data.shape[0]
    slice_slider.max = n_slices - 1
    slice_slider.value = n_slices // 2
    print(f"Volume có {n_slices} lát cắt.")

    # ====== PARAMS (đặt cố định ở đây cho khỏi lỗi biến) ======
    hu_emph = -950
    min_ratio = 0.13

    # 🔹 Detect emphysema
    emphy_hits = detect_emphysema_slices(data, hu_emph=hu_emph, min_ratio=min_ratio)
    # emphy_hits = [(i, ratio, low_count, lung_count), ...]
    

    # sort theo ratio giảm dần
    emphy_hits = sorted(emphy_hits, key=lambda x: x[1], reverse=True)

    SLICE_MIN, SLICE_MAX = 100, 200
    emphy_hits = [t for t in emphy_hits if SLICE_MIN <= int(t[0]) <= SLICE_MAX]

    print(f"Tìm được {len(emphy_hits)} lát nghi ngờ emphysema (slice {SLICE_MIN}-{SLICE_MAX}).")


    # ✅ Fill dropdown "Slice nghi ngờ"
    if len(emphy_hits) > 0:
        sus_slice_dropdown.options = [
            (f"Slice {i} ({r*100:.1f}% voxel < {hu_emph}HU | {low}/{lung})", int(i))
            for (i, r, low, lung) in emphy_hits
        ]
        sus_slice_dropdown.value = int(emphy_hits[0][0])
        sus_slice_dropdown.disabled = False
        current_volume_data["emphy_slices"] = [int(i) for (i, _, _, _) in emphy_hits]
    else:
        sus_slice_dropdown.options = [("Không có slice nghi ngờ", None)]
        sus_slice_dropdown.value = None
        sus_slice_dropdown.disabled = True
        current_volume_data["emphy_slices"] = []

    # refresh hình theo slice hiện tại
    update_slice_display(None)



def update_slice_display(change):
    data = current_volume_data["data"]
    if data is None:
        return
    
    idx = slice_slider.value
    hu_min = hu_min_slider.value
    hu_max = hu_max_slider.value

    slice_2d = data[idx, :, :]

    # Segment phổi cho lát hiện tại
    lung_mask = segment_lungs_hu(slice_2d)
    current_volume_data["last_lung_mask"] = lung_mask  # lưu lại, dùng lúc save

    # Chỉ giữ phổi, nền ngoài = hu_min (ví dụ -1000)
    slice_lung_only = np.where(lung_mask, slice_2d, hu_min)

    slice_norm = normalize_slice(slice_lung_only, hu_min=hu_min, hu_max=hu_max)

    out.clear_output(wait=True)
    with out:
        plt.figure(figsize=(6, 6))
        plt.imshow(np.flipud(slice_norm), cmap="gray")
        plt.axis("off")
        plt.title(f"Slice {idx} | HU[{hu_min}, {hu_max}] (lung only)")
        plt.show()
    
    info_label.value = (
        f"Volume: {current_volume_data['path'].name} | "
        f"Slice: {idx} | HU[{hu_min}, {hu_max}]"
    )


# 🔹 Khi chọn 1 slice nghi ngờ trong dropdown -> cập nhật slider slice
def on_sus_slice_change(change):
    if change["name"] == "value" and change["new"] is not None:
        slice_slider.value = int(change["new"])

sus_slice_dropdown.observe(on_sus_slice_change, names="value")

volume_dropdown.observe(on_volume_change, names="value")
slice_slider.observe(update_slice_display, names="value")
hu_min_slider.observe(update_slice_display, names="value")
hu_max_slider.observe(update_slice_display, names="value")


In [46]:
from ipywidgets import Button

btn_lung = Button(description="Preset Lung", button_style="info")
btn_emph = Button(description="Preset Emphysema", button_style="warning")
btn_mediast = Button(description="Preset Mediastinal", button_style="primary")

def set_lung(b):
    hu_min_slider.value = -1000
    hu_max_slider.value = 200

def set_emph(b):
    # hẹp hơn cho emphysema: vùng phổi sáng hơn, dễ thấy vùng khí phế thủng
    hu_min_slider.value = -950
    hu_max_slider.value = -500

def set_mediast(b):
    hu_min_slider.value = -100
    hu_max_slider.value = 300

btn_lung.on_click(set_lung)
btn_emph.on_click(set_emph)
btn_mediast.on_click(set_mediast)


In [47]:
def on_save_clicked(b):
    data = current_volume_data["data"]
    vol_path = current_volume_data["path"]
    
    if data is None or vol_path is None:
        info_label.value = " Chưa load volume nào."
        return
    
    idx = slice_slider.value
    label = label_buttons.value
    
    # 1) Lấy slice HU GỐC
    slice_2d = data[idx, :, :]           # HU gốc (float)

    # 2) Segment phổi (để lưu kèm lung_mask cho tiện dùng sau)
    lung_mask = current_volume_data.get("last_lung_mask")
    if lung_mask is None or lung_mask.shape != slice_2d.shape:
        lung_mask = segment_lungs_hu(slice_2d)

    # 3) Tên file raw (npz)
    vol_name = vol_path.stem  # bỏ đuôi .nii/.nii.gz
    out_filename = f"{vol_name}_z{idx:03d}.npz"
    out_path = OUT_DIR / out_filename

    # 4) Lưu HU gốc + lung_mask vào .npz (nén)
    np.savez_compressed(
        out_path,
        hu=slice_2d.astype(np.float32),
        lung_mask=lung_mask.astype(np.uint8),
    )

    # 5) Ghi dòng mới vào manifest
    global manifest_df
    new_row = {
        "volume_name": vol_name,
        "slice_index": idx,
        "label": label,
        "data_path": str(out_path.relative_to(BASE_DIR)),  # path tới file npz
    }
    manifest_df = pd.concat([manifest_df, pd.DataFrame([new_row])], ignore_index=True)
    manifest_df.to_csv(MANIFEST_PATH, index=False)
    
    info_label.value = f" Đã lưu slice gốc vào: {out_filename}"

save_button.on_click(on_save_clicked)




In [48]:
ui = VBox([
    volume_dropdown,
    sus_slice_dropdown,      
    slice_slider,
    hu_min_slider,
    hu_max_slider,
    HBox([btn_lung, btn_emph, btn_mediast]),
    HBox([label_buttons, save_button]),
    info_label,
    out,
])


In [49]:
display(ui)


In [50]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, IntSlider, VBox, HBox

# 1) Lấy danh sách file đã lưu trong OUT_DIR
npz_files = sorted([f for f in OUT_DIR.glob("*.npz")])

if not npz_files:
    print("Chưa có file .npz nào trong thư mục OUT_DIR.")
else:
    print(f"Tìm thấy {len(npz_files)} file đã lưu.")

# 2) Dropdown chọn file npz
npz_dropdown = Dropdown(
    options=[(f.name, f) for f in npz_files],
    description="Chọn file:",
    layout={"width": "400px"}
)

# 3) Slider chỉnh HU khi hiển thị
hu_min_slider2 = IntSlider(value=-1000, min=-1500, max=0, step=50, description="HU min")
hu_max_slider2 = IntSlider(value=200, min=-500, max=800, step=50, description="HU max")

out_npz = Output()

def display_npz(change):
    out_npz.clear_output()

    npz_path = npz_dropdown.value
    if npz_path is None:
        return
    
    # Load file
    data = np.load(npz_path)
    hu = data["hu"]
    lung_mask = data["lung_mask"].astype(bool)

    hu_min = hu_min_slider2.value
    hu_max = hu_max_slider2.value

    # window HU → normalize để xem
    hu_clipped = np.clip(hu, hu_min, hu_max)
    img_norm = (hu_clipped - hu_min) / (hu_max - hu_min + 1e-8)

    # lung-only
    lung_only = np.where(lung_mask, img_norm, 0)

    with out_npz:
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.title("HU gốc (window)")
        plt.imshow(img_norm, cmap='gray')
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Lung-only")
        plt.imshow(lung_only, cmap='gray')
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("Mask phổi")
        plt.imshow(lung_mask, cmap='gray')
        plt.axis("off")

        plt.show()

npz_dropdown.observe(display_npz, names="value")
hu_min_slider2.observe(display_npz, names="value")
hu_max_slider2.observe(display_npz, names="value")

display(VBox([npz_dropdown, HBox([hu_min_slider2, hu_max_slider2]), out_npz]))


Tìm thấy 579 file đã lưu.
